In [ ]:
import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
from chessutils import find_coeffs
from boardgen import chessboard

In [ ]:
from cornersdetect import cornersdetection

In [ ]:
PATH_TO_IMG = 'img'

In [ ]:
IMGSIZE = 480
MAXSHEAR = 0.15
MINSCALE = 0.5
NC = 5

HFIMGSIZE = IMGSIZE // 2

In [ ]:
figimgs = [f for f in os.listdir(PATH_TO_IMG) if f.split('_')[0]=='Chess']
figuresimgs = dict()
for f in figimgs:
    fn = f.split('_')[1].split('4')[0]
    img = cv2.imread(os.path.join(PATH_TO_IMG, f))
    figuresimgs[fn] = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

figs = ['p', 'b', 'n', 'r', 'q', 'k']
colors = ['d', 'l']

In [ ]:
parameters = {
    'numcell':8,
    'cellsize':45,
    'figures':figs,
    'colors':colors,
    'shear':MAXSHEAR,
    'scale':MINSCALE,
}

In [ ]:
def randomboardpoints(nc, params):
    boardsize = params['cellsize'] * params['numcell']
    mask = 255*np.ones((boardsize,boardsize), np.uint8)
    nb = params['numcell']*params['numcell']
    npt = np.random.randint(1,nc+1,size=nb)
    nzz = np.random.randint(nb, size=2)
    npt[nzz] = 0
    for i in range(nb):
        xp = i % params['numcell']
        yp = i // params['numcell']
        sz = npt[i]
        if (xp != 0) and (yp != 0) and sz > 0:
            mask[yp*params['cellsize']-sz:yp*params['cellsize']+sz,xp*params['cellsize']-sz:xp*params['cellsize']+sz] = 0
    return mask

In [ ]:
def boardmask(imgsize):
    boardimage, _, vecs = chessboard(figuresimgs, np.random.rand(13), imgsize, parameters)
    blank_mask = randomboardpoints(NC, parameters)

    img = Image.fromarray(blank_mask, 'L')
    coeffs = find_coeffs(
         vecs.reshape((4,2)),
         [(0, 0), (blank_mask.shape[1], 0), (blank_mask.shape[1], blank_mask.shape[0]), (0, blank_mask.shape[0])])

    img = img.transform((imgsize, imgsize), Image.PERSPECTIVE, coeffs, Image.NEAREST, fillcolor = 'white')
    return 1-np.asarray(img)/255

In [ ]:
def addpoints(img, num):
    xyadd = np.random.rand(num,3)
    for i in range(num):
        xc = int(img.shape[1]*xyadd[i,0])
        yc = int(img.shape[0]*xyadd[i,1])
        nc = int(NC*xyadd[i,2])+1
        img[yc-nc:yc+nc,xc-nc:xc+nc] = 1
    return

In [ ]:
bd = boardmask(IMGSIZE)
addpoints(bd, 2)
bz = 1-bd
zz = cornersdetection(bz)

In [ ]:
fig, axxes = plt.subplots(ncols=1, nrows=1, figsize=(6,6), sharex=True, sharey=True)
axxes.imshow(bz, cmap='gray', origin='lower')
for t in zz:
    axxes.add_patch(plt.Circle((t[0], t[1]), 5, color='b', fill=False))
fig.tight_layout()
plt.show()

In [ ]:
from cornersdetect import detectsdots

In [ ]:
xy = detectsdots(bz)
xy.shape

In [ ]:
fig, axxes = plt.subplots(ncols=1, nrows=1, figsize=(6,6), sharex=True, sharey=True)
axxes.imshow(bz, cmap='gray', origin='lower')
for t in xy:
    axxes.add_patch(plt.Circle((t[0], t[1]), 5, color='r', fill=False))
fig.tight_layout()
plt.show()

In [ ]:
from cornersdetect import detectshifts

In [ ]:
xyshift = detectshifts(bz)
xyshift